# Heisenberg model simulation - IBM Quantum challenge solution

In [1]:
import mitiq.interface  # Implements zero-noise extrapolation
from qiskit.opflow import Zero, One, I, X, Y, Z
from qiskit import Aer, QuantumCircuit, QuantumRegister, ClassicalRegister, execute, visualization, transpile, IBMQ
from qiskit.visualization import timeline_drawer # See the circuit timeline
from qiskit.quantum_info.operators import Operator # Helpful to implement randomized compiling
from qiskit.quantum_info import Pauli  
from qiskit.transpiler import PassManager, InstructionDurations   # Required to implement Dynamical Decoupling (DD)
from qiskit.transpiler.passes import ALAPSchedule, DynamicalDecoupling  # " " " "
from qiskit.circuit.library import XGate, CXGate, RZGate, IGate  # " " " "
from qiskit.providers.ibmq import RunnerResult  # To decode results obtained from Qiskit Runtime
from qiskit.providers.aer import AerSimulator
import matplotlib.pyplot as plt

# Import state tomography modules
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity

import numpy as np
import random  # Required for several quantum error mitigation techniques

### Implementation of exponential term of Pauli strings in a quantum circuit

The next cell implements an exponential term of a Pauli string of the form $U(t)=e^{-iP\frac{t}{2}}$, where the Pauli string is given by $P \in \{I,X,Y,Z\}^{\otimes n}$, being $n$ the number of qubits where the operator acts on.

Herein, Randomized Compiling is implemented to CNOT gates only. In the context of the Zero-Noise Extrapolation error mitigation technique, only the CNOT gates are folded (repeated). Both these techniques are only applied to the referred gates because they have an error two orders of magnitude larger than the single-qubit gate errors in IBMQ devices. Therefore, suppressing two-qubit gate errors is of central importance to obtain reliable results.

Function arguments:
- *qc* is the quantum circuit to apply $U(t)$.
- *gate* is the Pauli string given in *String* type. For example, $e^{-i Z\otimes X \frac{t}{2}}$ is given as $gate="ZX"$.
- *par* is the parameter *t* in $U(t)$.
- *qubits* represents the list of qubits the operator acts on. First qubit in the list is where the first Pauli gate is applied to ($Z$ in the example above).
- *scale* denotes how many times should the CNOT gates be folded (i.e. repeated). Note that this value must always be odd, otherwise we are changing the overall operator.

In [2]:
def pauli_exponential_random_comp(qc, gate, par, qubits, scale):
    # Remove any Identity gates in "gate".
    while ('I' in gate):
        del qubits[gate.index('I')]  # delete the qubit where the Identity gate is applied to.
        gate = gate.replace('I', '', 1)
    l = len(qubits)
    if l == 0:  # if only Identity gates in "gate" apply nothing.
        return
    for i in range(l): # Apply basis transformation
        if gate[i] == "X":
            qc.h(qubits[i])
        elif gate[i] == "Y":
            qc.sdg(qubits[i])
            qc.h(qubits[i])

    operator, op_next = assign_Pauli() # Obtain the Pauli Twirling gates to implement Randomized Compiling.
    qc.append(operator, qargs=qubits) # Apply Pauli Twirling
    for i in range(l - 1):
        qc.cx(qubits[i], qubits[i + 1])
    for s in range(round((scale - 1)/2)):   # Fold the CNOT gates.
        qc.barrier(qubits)
        for i in range(l - 1):
            qc.cx(qubits[i], qubits[i + 1])
        qc.barrier(qubits)
        for i in range(l - 1):
            qc.cx(qubits[i], qubits[i + 1])
    qc.append(op_next, qargs=qubits)     # Apply the inverse of Pauli Twirling.

    qc.rz(par, qubits[l - 1])   

    operator, op_next = assign_Pauli() # Obtain the Pauli Twirling gates to implement randomized compiling.
    qc.append(operator, qargs=qubits)    # Apply Pauli Twirling
    for i in range(l - 2, -1, -1):
        qc.cx(qubits[i], qubits[i + 1])
    for s in range(round((scale - 1)/2)):     # Fold the CNOT gates.
        qc.barrier(qubits)
        for i in range(l - 2, -1, -1):
            qc.cx(qubits[i], qubits[i + 1])
        qc.barrier(qubits)
        for i in range(l - 2, -1, -1):
            qc.cx(qubits[i], qubits[i + 1])
    qc.append(op_next, qargs=qubits)     # Apply the inverse of Pauli Twirling.
    for i in range(l):              # Apply the inverse of the basis transformation
        if gate[i] == "X": 
            qc.h(qubits[i])
        elif gate[i] == "Y":
            qc.h(qubits[i])
            qc.s(qubits[i])

### Pauli Twirling

The next function was used previously to apply Randomized Compiling to CNOT gates. This function does the following:
1. It samples an uniform distribution of all the $2$-qubit Pauli strings to be applied to qubits where the CNOT gate is implemented at. The resulting Pauli string is given as the variable *operator*.
2. Calculates the inverse of the Pauli Twirling technique (read the paper about Randomized Compiling for further details). Note that gates of the Clifford group (where the CNOT gate belongs to) if applied to a Pauli gate generate a different Pauli string. This resultant Pauli string is given as the variable *op_next*. The following calculation is performed on the fly when creating the circuits.

In [3]:
def assign_Pauli():
    op = []
    for i in range(2):
        random_n = random.randint(0, 3) # uniform sampling
        if random_n == 0:
            op.append(Operator(Pauli(label='X')))
        elif random_n == 1:
            op.append(Operator(Pauli(label='Y')))
        elif random_n == 2:
            op.append(Operator(Pauli(label='Z')))
        else:
            op.append(Operator(Pauli(label='I')))
    # Calculate the inverse of the sampled 2-qubit Pauli Twirling gate
    operator = op[0].tensor(op[1])
    op_next = Operator(CXGate()).compose(operator.compose(CXGate()))
    return operator, op_next

### State initialization
Initialize the three qubits in the state $|110\rangle$ as requested by IBM.

In [4]:
def initialize_qubits(qc):
    qc.x([0,1])

### Implementation of the first-order Trotter-Suzuki formula (one iteration)
I denote the three qubits to be used in the simulation by the indexes 0, 1 and 2. 
Remember $J=1$ for all Pauli strings in the Hamiltonian and that the $R_{Z}(\theta)$ gate in Qiskit is implemented as $e^{-iZ\frac{\theta}{2}}$, hence we need to define $\theta=2 \Delta t$, being $\Delta t=t/N$ the Trotter time-step ($t$ is the time the model is simulated for and $N$ is the number of Trotter iterations).

In [5]:
def one_iteration_Trotter(qc,delta_t, scale):
    for i in range(2):
        pauli_exponential_random_comp(qc, "XX", 2*delta_t, [i,i+1], scale)
    for i in range(2):
        pauli_exponential_random_comp(qc, "YY", 2*delta_t, [i,i+1], scale)
    for i in range(2):
        pauli_exponential_random_comp(qc, "ZZ", 2*delta_t, [i,i+1], scale)

Check the correctness of the circuits.

In [6]:
N=2
final_time = np.pi         # specified by IBM
delta_t = final_time / N 
scale = 1
qc = QuantumCircuit(3, 1)
initialize_qubits(qc)
for l in range(N):
    one_iteration_Trotter(qc,delta_t,scale)
qc.draw(output = 'mpl')

/Users/joseguimaraes/opt/anaconda3/lib/python3.7/site-packages/qiskit/quantum_info/operators/symplectic/pauli.py:197: DeprecationWarning: Initializing Pauli from `Pauli(label=l)` kwarg is deprecated as of version 0.17.0 and will be removed no earlier than 3 months after the release date. Use `Pauli(l)` instead.
  base_z, base_x, base_phase = self._from_label_deprecated(label)


## Assemble everything
Create the circuits with quantum error mitigation and transpile them to be used in a real quantum computer.
Note that the real qubits to be used in the simulation have been already specified by IBM (the $5,3,1$ qubits in the *ibmq_jakarta*).

#### Variables:
- *N* is the number of Trotter iterations.
- *n_runs_RC* is the number of different circuits to be created in order to perform Randomized Compiling. Note that each circuit is executed with different Pauli Twirling gates applied to each CNOT gate yielding a total of *n_runs_RC* different circuits. This is the last process of Randomized Compiling which allows us to suppress coherent errors (without this, it has been shown that Zero-Noise Extrapolation works poorly).
- *shots* is the number of shots for each circuit execution.
- *scale_factors_ZNE* is a list of how many times should the CNOT gates in the circuits be folded. For example, if we have [1,3], then each circuit of the *n_runs_RC* existent circuits, will have all of its CNOT gates folded 1 (i.e. no folded gates) and 3 times. This increases the number of circuits to be executed to $2$ $\times$ *n_runs_RC*.

In [7]:
def generate_Trotter_and_Tomography (N, n_runs_RC, delta_t, scale_factors_ZNE):
    qc_list = []
 
    for scale in scale_factors_ZNE:     # Create n_runs_RC circuits for each ZNE scaling factor.
        for j in range(n_runs_RC):
            qc = QuantumCircuit(3)
            initialize_qubits(qc)
            for l in range(N):
                one_iteration_Trotter(qc, delta_t, scale)
                
            # Generates 27 (3^3) state tomography circuits to evaluate fidelity of simulation
            qc = state_tomography_circuits(qc, [0,1,2])
            qc_list.extend(qc)         # add to a list in order to create a job with all these circuits
    
    return qc_list;

## Simulation


We are limited by 32000 shots per circuit execution and a maximum of 300 circuits in a batch of circuits (Qiskit Runtime). The number of circuits executions in a batch for our simulation is: $2 \times 27\times$*n_runs_RC*$=54 \times$ *n_runs_RC*. We may need to send several jobs in order to execute several circuits to implement Randomized Compiling, as follows.

In [8]:
from qiskit.providers.aer import QasmSimulator
from qiskit.tools.monitor import job_monitor

# Get backend for experiment
IBMQ.load_account()

#provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')

provider = IBMQ.get_provider(
    hub='ibm-q-minho',
    group='academicprojects',
    project='quantalab'
)

jakarta = provider.get_backend('ibmq_jakarta')
guadalupe = provider.get_backend('ibmq_guadalupe')


# Simulated backend based on ibmq_jakarta's device noise profile
sim_noisy_jakarta = QasmSimulator.from_backend(jakarta)

# Noiseless simulated backend
sim = Aer.get_backend('aer_simulator_statevector')

## Implementation of Dynamical Decoupling (DD)


In [15]:
def execute_circuit_batch (qc_list, backend, shots, initial_layout):
    job = provider.run_circuits(qc_list, backend_name=backend.name(), shots=shots, optimization_level=1,
                                 initial_layout=initial_layout, measurement_error_mitigation=True)
    
    return job


def execute_circuit_batch_with_dd (qc_list, backend, shots, initial_layout):
    
    # Create job using Qiskit Runtime by sending a batch of quantum circuits to be executed.
    # It transpiles the circuit with optimization_level = 1, maps the virtual qubits to real qubits and
    # applies measurement error mitigation (see references in Qiskit for further details about this technique).
    
    # DYNAMICAL DECOUPLING 
    
    print("Transpiling circuits...")     # First, we need to transpile the circuits
    qc_list = transpile(qc_list,backend=backend,optimization_level=1,initial_layout=initial_layout)
    print("Adding dynamic decoupling sequences...")
    durations = InstructionDurations.from_backend(backend) # Get the durations of the gates implementation
    
    #XX
    dd_sequence = [XGate(),XGate()] 
    d=1/2
    spacing = [d/2,d,d/2]
    # XY4
    
    # define the XY4 sequence, i.e. (X-Y-X-Y, which equals the Identity gate)
    #dd_sequence = [XGate(),RZGate(-np.pi),XGate(),XGate(),RZGate(-np.pi),XGate()] 
    #d=1/4
    #spacing = [d/2,d/2,d/2,d,d/2,d/2,d/2]   # define the interval of time (i.e. d/2 or d) between DD gates. 
                                            # Do not change this (the pulses must be equally spaced).
                                            # This interval of time is relative to the duration that 
                                            # the qubit stays idle, i.e. no gate applied to it. Therefore,
                                            # sum(spacing)=1, i.e. the sequence must span the entire
                                            # interval of time that the qubit stays idle
    # XY4 x 2
    #dd_sequence = [XGate(),RZGate(-np.pi),XGate(),XGate(),RZGate(-np.pi),XGate(),XGate(),RZGate(-np.pi),XGate(),XGate(),RZGate(-np.pi),XGate()] 
    #d=1/8
    #spacing = [d/2,d/2,d/2,d,d/2,d/2,d,d/2,d/2,d,d/2,d/2,d/2]
    
    spacing[-1] = spacing[-1]+1-sum(spacing)    # guarantee that sum(spacing)=1 (required)
    pm = PassManager([ALAPSchedule(durations),
         DynamicalDecoupling(durations, dd_sequence,spacing=spacing)]) # create the DD sequence applied to all qubits
    
    qc_list = pm.run(qc_list) # apply the DD sequences to our circuits
    
    #timeline_drawer(qc_list[0],filename='timeline')
     # RUN circuits (scheduling method must be set to 'alap' and optimization_level to 0 here).
        # optimization level must be 0 here because the DD sequence equals the Identity gate. 
        # Hence, any performed optimization will remove the DD sequences. Notice that the circuits are optimized
        # because we transpiled them before applying the DD sequences.
        # scheduling method = alap equals "as late as possible" apply the gates. This is required.
    job = provider.run_circuits(
            qc_list, backend_name='ibmq_guadalupe', shots=shots, optimization_level=0,
            measurement_error_mitigation=True, transpiler_options={'scheduling_method': 'alap'})
    return job

In [16]:
N=4                         # N=4 is the minimum Trotter iterations in order for our work to be considered.
n_runs_RC = 10             # I think 30 is ok. Choose a multiple of 10.
shots = 16000
scale_factors_ZNE = [1,3]   # These folding values should work fine.

final_time = np.pi         # specified by IBM
delta_t = final_time / N 

#backend = sim
#backend = sim_noisy_jakarta
#backend = jakarta
backend = guadalupe

#initial_layout = [5,3,1]    # assign each qubit in our simulation to a real qubit.
                            # real qubit = value of the element of the list. 
                            # simulation qubit = index of the element of the list.
        
initial_layout = [1,4,7]

number_reps = 1
jobs_list = []

qc_list = generate_Trotter_and_Tomography (N, n_runs_RC, delta_t, scale_factors_ZNE)

In [17]:
for i in range(number_reps):
    # each job has 10x27 jobs. Maximum in each job are 300 circuits.
    for j in range(n_runs_RC//5):
        job = execute_circuit_batch_with_dd (qc_list[j*10*27:(j+1)*10*27], backend, shots, initial_layout)
        print('Job ID', job.job_id())
        jobs_list.append(job.job_id())

np.savez("jobs_list", jobs_list=jobs_list)

Transpiling circuits...
Adding dynamic decoupling sequences...
Job ID c8ktcus6e325qt1s5sbg
Transpiling circuits...
Adding dynamic decoupling sequences...
Job ID c8ktd9b6ecfgg38dvag0


Get back later and obtain the results of the simulations.

In [19]:
import copy

def state_tomography(result, st_qcs):
    
    # The expected final state; necessary to determine state tomography fidelity
    target_state = (One^One^Zero).to_matrix()  # DO NOT MODIFY (|q_5,q_3,q_1> = |110>)
    # Fit state tomography results
    tomo_fitter = StateTomographyFitter(result, st_qcs)
    rho_fit = tomo_fitter.fit(method='lstsq')
    # Compute fidelity
    fid = state_fidelity(rho_fit, target_state)
    return fid

def combine_results (results):

    returning_result = copy.deepcopy (results [0])
    counts = {'0x0': 0,'0x1': 0,'0x2': 0,'0x3': 0,'0x4': 0,'0x5': 0,'0x6': 0,'0x7': 0}

    for result in results:
        for key in result.data.counts.keys ():
            counts [key] += result.data.counts [key]

    returning_result.data.counts = counts
    returning_result.shots = results [0].shots * len (results)
  
    return returning_result

def combine_job_results (results_ZNE_scale,results_jobs):
    if results_ZNE_scale == []:
        results_ZNE_scale = results_jobs[0]
    else:
        for i_tomography,result in enumerate(results_jobs[0].results):
            for key in result.data.counts.keys ():
                results_ZNE_scale.results[i_tomography].data.counts[key] += result.data.counts[key]
                results_ZNE_scale.results[i_tomography].shots += result.shots
    return results_ZNE_scale


def states_random_compiling(result_list, qc_list):
    i = 0
    fids_random = []
    
    results = result_list.results
    result_metadata = copy.deepcopy (result_list)
    
    result_metadata.results = []
    j = 0
    # For each tomography circuit...
    while j < 27:
        random_compilation_results = []
        i =  0
        # Collect each respective tomography result, e.g. 'XXX', from each of the random compilation circuits
        while i < len (results)/27:
            random_compilation_results.append (results [i*27+j])
            i+=1
        
        # Combine all results for a circuit tomography, simply by joining the sampling results, 
        # see function combine results
        result_metadata.results.append (combine_results (random_compilation_results))
        j+=1

    return result_metadata

def states_zero_noise_extrapolation (job_list, qc_list, factors,shots):
    results_ZNE = [[],[]]
    l_jobs = len(job_list)
    for ind_job , job in enumerate(job_list):
        results_job = []
        # The result list will contain the results for all noise factors
        result_list = provider.runtime.job(job)
        #results = result_list.result()
        #result_metadata = copy.deepcopy (result_list)
        probs_list = result_list.result(decoder=RunnerResult).get_quasiprobabilities()
        result_list = result_list.result(decoder=RunnerResult)

         # Manipulation of results for qiskit measurement error mitigation (comment if no measurement error mitigation)
        #for i in range(len(probs_list)):
         #   distrib = probs_list[i].nearest_probability_distribution()
          #  for key in distrib:
           #     result_list.results[i].data.counts[hex(key)] = round(distrib[key]*shots) # counts must be a integer number
         #---------- Comment until here

        results = result_list.results
        result_metadata = copy.deepcopy(result_list)
        # And the number of elements per factor is the same
        number_of_factors = len (factors)
        elements_per_factor = len (results)//number_of_factors
        
        result_metadata.results = results

        results_job.append(states_random_compiling(result_metadata, qc_list))
        
        if ind_job<l_jobs//2: # index smaller than l_jobs//2 is for scale_factor = 1
            results_ZNE[0] = combine_job_results(results_ZNE[0],results_job)
        else:                 # index greater or equal to l_jobs//2 is for scale_factor = 3
            results_ZNE[1] = combine_job_results(results_ZNE[1],results_job)
    
    # All that  remains is to extrapolate the results for each tomography set, using the different factors
    zne_tomography_results = copy.deepcopy (results_ZNE [0])
    tomography_i = 0
    #print(results_ZNE)

    # For each tomography measurement set
    while tomography_i < len (results_ZNE [0].results):

        observables_extrapolated = {'0x0': 0,'0x1': 0,'0x2': 0,'0x3': 0,'0x4': 0,'0x5': 0,'0x6': 0,'0x7': 0}

        for observable in observables_extrapolated.keys(): 
            observable_counts = []
            for zne_results in results_ZNE:
                observable_counts.append (zne_results.results [tomography_i].data.counts [observable])
            
            # Do the zero noise extrapolation for each observable, using the statistics obtained for each factor
            zero_limit_value = mitiq.zne.inference.ExpFactory.extrapolate(
            scale_factors = factors, exp_values = observable_counts,asymptote = 0)
            # asymptote = 0 because the other states go to 0 in the infinite noise limit (depolarizing noise)
                
            observables_extrapolated [observable] = round(zero_limit_value) # counts must be a integer number
        
        obs_extrapolated_sum = sum(observables_extrapolated[key] for key in observables_extrapolated)
        zne_tomography_results.results [tomography_i].data.counts = copy.deepcopy (observables_extrapolated)
        zne_tomography_results.results [tomography_i].data.shots = obs_extrapolated_sum
        tomography_i += 1
    print ("---------")
    #print(zne_tomography_results)
    return zne_tomography_results    

# Compute tomography fidelities for each repetition

#jobs_list = ["c8h0pr5p1qlipf6ugedg","c8h0q323dihfro03dmk0","c8h0qai3dihfro03doqg","c8h0ql0r7l5c7vuuebcg","c8h0r2lp1qlipf6ugpj0","c8h0rca3dihfro03e2dg"]

with np.load("jobs_list.npz") as data:
    jobs_list = data["jobs_list"]
results_processed = states_zero_noise_extrapolation (jobs_list, qc_list, scale_factors_ZNE,shots)
fid = state_tomography(results_processed, qc_list[:27])

print('state tomography fidelity = {:.4f} \u00B1 {:.4f}'.format(np.mean(fid), np.std(fid)))

---------
state tomography fidelity = 0.7866 ± 0.0000


# Fidelity on noisy classical simulator jakarta

16000 shots always per circuit

No RC, no ZNE:

    N = 4 : 0.5112
    
    N = 5 : 0.2624
    
    N = 6 : 0.1575

RC (20 circuits), no ZNE, N = 4: 0.5132

RC (30 circuits), no ZNE, N = 4: 0.5127

No RC, Richardson ZNE, N = 4: 0.6595    (In real quantum computers would not work... but we should try)

No RC, Linear ZNE, N = 4: 0.6654

No RC, Exponential ZNE, N = 4: 0.6641

RC (20 circuits), Richardson ZNE, N = 4: 0.6567

# Real quantum computer (ibmq_guadalupe):

## No RC and N=4:

### With measurement error mitigation, no DD:

Exponential extrapolation: 0.5337

Richardson extrapolation: 0.5366

No extrapolation: 0.3111

### With measurement error mitigation, with DD (all qubits)

Exponential extrapolation: 0.6267

Richardson extrapolation: 0.4738

No extrapolation: 0.4738

### Without measurement error mitigation, no DD:

Exponential extrapolation: 0.5913

Richardson extrapolation: 0.4447

No extrapolation: 0.2782

### Without measurement error mitigation, with DD (all qubits)

Exponential extrapolation: 0.7066

Richardson extrapolation: 0.4022

No extrapolation: 0.4022

# Another experiment

## With RC (30 circuits per scale factor), DD (all qubits) and N=4:

### with measurement error mitigation:

Exponential extrapolation: 0.6805

Richardson extrapolation: 0.5113

### no measurement error mitigation:

Exponential extrapolation: 0.6788

Richardson extrapolation: 0.4261

## With RC (10 circuits per scale factor), DD (all qubits) and N=4:

### with measurement error mitigation:

Exponential extrapolation: 0.6487

Richardson extrapolation: 0.4621

### no measurement error mitigation:

Exponential extrapolation: 0.6507

Richardson extrapolation: 0.3968

# Another experiment

### Exp ZNE, RC (10 circuits), N=4:

With DD XY4, with measurement error mitigation : 0.6566

With DD XY4x2, with measurement error mitigation : 0.6295

With DD XX, with measurement error mitigation : 0.7604

With DD XX, NO measurement error mitigation : 0.7866

Things to consider:

Exponential ZNE is by far the best mitigation method. DD improves the results. Measurement error mitigation may decrease the fidelity if we use Exponential ZNE with no RC. 